In [1]:

import sys, os

sys.path.append(os.path.abspath(os.path.join('../scripts')))


In [ ]:
import importlib
import fetch
importlib.reload(fetch)
from fetch import FetchData

In [3]:
from shapely.geometry import shape, GeometryCollection, Polygon,Point
import geopandas as gpd

In [4]:
import pyproj

In [4]:
MINX, MINY, MAXX, MAXY = [-93.756155, 41.918015, -93.747334, 41.921429]
polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))

grid = gpd.GeoDataFrame([polygon], columns=["geometry"])
grid.set_crs(epsg=4326, inplace=True)


,geometry
0,"POLYGON ((-93.75616 41.91801, -93.75616 41.921..."


In [20]:

data=FetchData("IA_FullState").fetch_elevation(grid,"EPSG:4326")

In [6]:
data[0].describe()

,elevation
count,189060.000000
mean,316.202441
std,2.537666
min,306.290000
25%,315.080000
50%,317.150000
75%,317.940000
max,319.760000


In [7]:

import matplotlib.pyplot as plt
%matplotlib widget
FetchData("IA_FullState").visualize3D(data[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
%matplotlib widget
FetchData("IA_FullState").visualize2D(data[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
data[0].head()

,elevation,geometry
0,310.10,POINT (-93.75605 41.91804)
1,310.56,POINT (-93.75566 41.91819)
2,310.40,POINT (-93.75605 41.91864)
3,311.59,POINT (-93.75606 41.91908)
4,312.16,POINT (-93.75586 41.91923)


In [18]:
standardized=FetchData("IA_FullState").standardize(data[0],1)

D:\Users\same\anaconda3\envs\pdalworkshop\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [19]:
%matplotlib widget
FetchData("IA_FullState").visualize3D(standardized)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# import geoplot
# import geoplot.crs as gcrs
# geoplot.kdeplot(data[0],shade=True, cmap='Spectral_r',projection=gcrs.WebMercator(), shade_lowest=True)

In [ ]:
# import numpy as np
# x=data[0].geometry.x
# y=data[0].geometry.y
# z = np.array(data[0].elevation)

# z = z.reshape((len(x), len(y)))
# plt.contour(x,y,z, colors='black');

In [10]:
import importlib
import fetch_without_region
importlib.reload(fetch_without_region)

<module 'fetch_without_region' from 'd:\\same\\Documents\\10Acadamy\\Week6\\AgriTech\\scripts\\fetch_without_region.py'>

In [11]:
from fetch_without_region import FetchDataRegionLess

In [8]:
import geopandas as gpd

In [12]:
MINX, MINY, MAXX, MAXY = [-93.756155, 41.918015, -93.747334, 41.921429]
polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))

grid = gpd.GeoDataFrame([polygon], columns=["geometry"])
grid.set_crs(epsg=4326, inplace=True)

,geometry
0,"POLYGON ((-93.75616 41.91801, -93.75616 41.921..."


In [14]:
FetchDataRegionLess().fetch_elevation(grid,4326)

In [ ]:
plt.text